In [1]:
import import_ipynb
import Classification_Function 
import tensorflow as tf
from imp import reload
reload(Classification_Function)
from Classification_Function import *


file_path="MASK_File/마스크_20_1030.xlsx"
train_df_origin=pd.read_excel(file_path)
train_df=train_df_origin[["goods_nm"]]

train_df=train_text_preprocessing(train_df)

stop=pd.read_table("stopwords_korean.txt",header=None)
stopwords=stop[0].tolist()


print("tokenizer")
print("============================")
X_train=tokenizer(train_df,stopwords)
X_dict={}
print("Vocab indexing")
print("============================")
X_train,X_dict=vocab_indexing(X_train)  
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

X_train = pad_sequences(X_train, maxlen = 30)

from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


model = load_model("train_model_adam_1023_Mask.h5")



print("Prediction")
print("============================")
with tf.device('/cpu:0'):
    
    predict=model.predict(X_train)
    predict_labels = np.argmax(predict, axis=1)


import pandas as pd

label_table=pd.read_excel("마스크 라벨.xlsx")
Label=pd.DataFrame(label_table)
Label=dict(zip(Label["KF"].tolist(),Label["Label"].tolist()))


return_label={}
for i in tqdm(range(len(label_table))):
    return_label.setdefault(label_table.iloc[i]['Label'],label_table.iloc[i]['KF'])
    

goods_name=train_df['goods_nm']
goods=goods_name.tolist()

predict_series={"상품명 ":[], "예측한 코드 " : []}
predict_Table=pd.DataFrame(predict_series)
num=0

def largest_indices(ary, n):
    """Returns the n largest indices from a numpy array."""
    flat = ary.flatten()
    indices = np.argpartition(flat, -n)[-n:]
    indices = indices[np.argsort(-flat[indices])]
    return np.unravel_index(indices, ary.shape)


def return_label_2(label):
    result=[]
    for i in range(len(label[0])):
        for k,v in Label.items():
            if v==label[0][i]:
                result.append(k)
    return result

print("Excel 생성중")


predict_series={ "예측한 라벨 1 " : [], "예측 확률 1 ": [], "예측한 라벨 2 " : [], "예측 확률 2 ": [], "예측한 라벨 3 " : [], "예측 확률 3 ": [], }
predict_Table=pd.DataFrame(predict_series)
print("============================")
for i in tqdm(range(len(predict_labels))):   
    num=num+1
    new_data={"예측한 라벨 1 " : return_label_2(largest_indices(predict[i],3))[0],
              "예측 확률 1 " :predict[i][largest_indices(predict[i],3)][0], 
              "예측한 라벨 2 " : return_label_2(largest_indices(predict[i],3))[1],
              "예측 확률 2 " : predict[i][largest_indices(predict[i],3)][1],
              "예측한 라벨 3 " : return_label_2(largest_indices(predict[i],3))[2],
              "예측 확률 3 " : predict[i][largest_indices(predict[i],3)][2],
              
             }
    predict_Table=predict_Table.append(new_data,ignore_index=True)

result=pd.concat([train_df_origin,predict_Table],axis=1)
output_path=file_path.replace(".xlsx","_result.xlsx")
csv_path=file_path.replace(".xlsx","_result.csv")
result.to_excel(output_path)
result.to_csv(csv_path,encoding="UTF-8")


print("작업 완료")

ModuleNotFoundError: No module named 'import_ipynb'